In [63]:
!pip install genism

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement genism (from versions: none)
ERROR: No matching distribution found for genism


In [64]:
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import re
import nltk 
import gensim
from gensim.models import Word2Vec
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [66]:
df = pd.read_csv('Train.csv')

In [67]:
df

,tweet,label,id
0,@USER She should ask a few native Americans wh...,OFF,0
1,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,1
2,Amazon is investigating Chinese employees who ...,NOT,2
3,"@USER Someone should'veTaken"" this piece of sh...",OFF,3
4,@USER @USER Obama wanted liberals &amp; illega...,NOT,4
...,...,...,...
13235,@USER Sometimes I get strong vibes from people...,OFF,13235
13236,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,13236
13237,@USER And why report this garbage. We don't g...,OFF,13237
13238,@USER Pussy,OFF,13238


**Checking for missing values in the dataset**

In [68]:
df.isnull().sum()

tweet    0
label    0
id       0
dtype: int64

**converting label values to numbers 0, 1**

In [69]:
def convertlabel(st):
  if st == 'OFF':
    return 0
  elif st == 'NOT':
    return 1

In [70]:
for i in range(len(df['label'])):
  df['label'][i] = convertlabel(df['label'][i])

<ipython-input-70-085057400cad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i] = convertlabel(df['label'][i])


**cleaning each tweet (stemming and removing stopwords, hashtags, urls etc)**

In [71]:
stop_words = stopwords.words('english')
st = PorterStemmer()

In [72]:
def stemtweet(text):                #Stemming
  text = word_tokenize(text)

  for x in range(len(text)):
    text[x] = st.stem(text[x])

  return " ".join(x for x in text)

In [73]:
def remstopwords(text):                #Remove Stop Words
  text = text.split()
  list = []
  for x in text:
    if x not in stop_words:
      list.append(x)
  
  newtext = " ".join(word for word in list)
  return newtext

In [74]:
def cleantweet(text):
  text = re.sub("(@|#)[A-Za-z0-9_]+" , "", text)       #Remove usernames/Hashtags
  text = re.sub(r'(www.)|http\S+', '', text)                      #Remove URLs
  text = re.sub('[()!?]', ' ', text)                    #Remove Punctuation marks
  text = re.sub('\[.*?\]',' ', text)
  text = re.sub("[^a-z0-9]"," ", text)                 #Remove irrelevant characters
  
  text = remstopwords(text)

  text = stemtweet(text)

  return text

In [75]:
df['tweet'] = df['tweet'].apply(lambda x: cleantweet(x.lower()))

In [76]:
df

,tweet,label,id
0,ask nativ american take,0,0
1,go home drunk url,0,1
2,amazon investig chines employe sell intern dat...,1,2
3,someon vetaken piec shit volcano,0,3
4,obama want liber amp illeg move red state,1,4
...,...,...,...
13235,sometim get strong vibe peopl man vibe ten mil...,0,13235
13236,benidorm creamfield maga shabbi summer,1,13236
13237,report garbag give crap,0,13237
13238,pussi,0,13238


**Forming a new Dataset with X = vector, y = label**

In [77]:
newdf = df[['tweet', 'label']]

In [78]:
newdf['tweet']

0                                  ask nativ american take
1                                        go home drunk url
2        amazon investig chines employe sell intern dat...
3                         someon vetaken piec shit volcano
4                obama want liber amp illeg move red state
                               ...                        
13235    sometim get strong vibe peopl man vibe ten mil...
13236               benidorm creamfield maga shabbi summer
13237                              report garbag give crap
13238                                                pussi
13239                                               vs url
Name: tweet, Length: 13240, dtype: object

In [79]:
newdf

,tweet,label
0,ask nativ american take,0
1,go home drunk url,0
2,amazon investig chines employe sell intern dat...,1
3,someon vetaken piec shit volcano,0
4,obama want liber amp illeg move red state,1
...,...,...
13235,sometim get strong vibe peopl man vibe ten mil...,0
13236,benidorm creamfield maga shabbi summer,1
13237,report garbag give crap,0
13238,pussi,0


In [80]:
X = newdf['tweet']
y = newdf['label']

In [81]:
X

0                                  ask nativ american take
1                                        go home drunk url
2        amazon investig chines employe sell intern dat...
3                         someon vetaken piec shit volcano
4                obama want liber amp illeg move red state
                               ...                        
13235    sometim get strong vibe peopl man vibe ten mil...
13236               benidorm creamfield maga shabbi summer
13237                              report garbag give crap
13238                                                pussi
13239                                               vs url
Name: tweet, Length: 13240, dtype: object

In [82]:
X.shape

(13240,)

**Converting each tweet into a vector based on the occurences of all the unique words in corpus**

Bag of Words

In [83]:
vectorizer = CountVectorizer(max_features = 3266)

X = vectorizer.fit_transform(X)

TF IDF

In [20]:
vectorizer = TfidfVectorizer(max_features = 3266)

X = vectorizer.fit_transform(X)

In [84]:
X.shape

(13240, 3266)

In [85]:
y = y.to_numpy()

In [86]:
y = y.astype('int64')

In [87]:
newdf

,tweet,label
0,ask nativ american take,0
1,go home drunk url,0
2,amazon investig chines employe sell intern dat...,1
3,someon vetaken piec shit volcano,0
4,obama want liber amp illeg move red state,1
...,...,...
13235,sometim get strong vibe peopl man vibe ten mil...,0
13236,benidorm creamfield maga shabbi summer,1
13237,report garbag give crap,0
13238,pussi,0


**Splitting the dataset**

In [88]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

Training on Train data and evaluating on Validation data

**Train ANN**

In [89]:
model1 = MLPClassifier(activation = 'logistic')
model1.fit(X_train, y_train)
ypred = model1.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, ypred))

Accuracy: 0.7077039274924471


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


**Train SVM**

In [106]:
model2 = SVC()
model2.fit(X_train, y_train)
ypred = model2.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, ypred))

Accuracy: 0.7549093655589124


**Train Logistic Function**

In [107]:
model3 = LogisticRegression()
model3.fit(X_train, y_train)
ypred = model3.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, ypred))

Accuracy: 0.7651057401812689


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Ensemble Training (Random Forest)**

In [108]:
model4 = RandomForestClassifier()
model4.fit(X_train, y_train)
ypred = model4.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, ypred))

Accuracy: 0.7658610271903323


In [94]:
model5 = GaussianNB()
model5.fit(X_train.todense(), y_train)
ypred = model5.predict(X_valid.todense())
print('Accuracy:', accuracy_score(y_valid, ypred))

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Accuracy: 0.4142749244712991


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


**Working On Test Data**

In [95]:
testdf = pd.read_csv('Test.csv')

In [96]:
testdf

,tweet,id
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,0
1,"#ConstitutionDay is revered by Conservatives, ...",1
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,2
3,#Watching #Boomer getting the news that she is...,3
4,#NoPasaran: Unity demo to oppose the far-right...,4
...,...,...
855,#DespicableDems lie again about rifles. Dem Di...,855
856,#MeetTheSpeakers 🙌 @USER will present in our e...,856
857,3 people just unfollowed me for talking about ...,857
858,#WednesdayWisdom Antifa calls the right fascis...,858


In [97]:
testdf.isnull().sum()

tweet    0
id       0
dtype: int64

In [98]:
testdf['tweet'] = testdf['tweet'].apply(lambda x: cleantweet(x.lower()))

In [99]:
testdf

,tweet,id
0,democrat support antifa muslim brotherhood ms1...,0
1,rever conserv hate progress socialist democrat...,1
2,first reduc ca url,2
3,get news still parol alway make smile final tr...,3
4,uniti demo oppos far right enough enough url,4
...,...,...
855,lie rifl dem distort law push gun control kava...,855
856,present event oiw 2018 finpact global impact f...,856
857,3 peopl unfollow talk merlin sorri im still co...,857
858,antifa call right fascist realiti left follow ...,858


In [102]:
newX = testdf['tweet']

In [103]:
newX = vectorizer.fit_transform(newX)

In [109]:
newpred = model4.predict(newX)

In [110]:
print('id,label')

for x in range(len(newpred)):
  print(str(x) + "," + str(newpred[x]))

id,label
0,0
1,0
2,1
3,0
4,1
5,1
6,0
7,1
8,1
9,1
10,1
11,0
12,1
13,1
14,1
15,1
16,0
17,1
18,1
19,1
20,0
21,0
22,0
23,0
24,1
25,0
26,1
27,0
28,0
29,1
30,1
31,1
32,0
33,0
34,1
35,1
36,1
37,1
38,0
39,1
40,1
41,1
42,1
43,0
44,0
45,1
46,1
47,1
48,0
49,0
50,1
51,1
52,1
53,1
54,0
55,1
56,1
57,0
58,1
59,1
60,0
61,0
62,1
63,0
64,0
65,0
66,0
67,1
68,0
69,1
70,1
71,0
72,1
73,0
74,0
75,0
76,1
77,0
78,0
79,0
80,1
81,1
82,1
83,0
84,0
85,1
86,1
87,0
88,1
89,0
90,1
91,0
92,1
93,1
94,1
95,1
96,0
97,1
98,1
99,1
100,0
101,1
102,1
103,1
104,0
105,1
106,0
107,1
108,1
109,0
110,0
111,1
112,1
113,0
114,0
115,0
116,1
117,1
118,1
119,1
120,0
121,1
122,0
123,1
124,1
125,0
126,1
127,1
128,1
129,1
130,0
131,0
132,0
133,1
134,1
135,1
136,1
137,1
138,0
139,1
140,1
141,0
142,1
143,1
144,1
145,0
146,1
147,0
148,1
149,1
150,1
151,0
152,0
153,1
154,1
155,0
156,0
157,0
158,1
159,1
160,1
161,0
162,1
163,0
164,1
165,0
166,1
167,0
168,1
169,1
170,1
171,0
172,0
173,1
174,1
175,1
176,1
177,0
178,1
179,0
180,0
181,1
182,1
183

In [ ]:
5020101001574